### Installing Required Packages

In [1]:
!pip install dataset

  Using cached dataset-1.4.5-py2.py3-none-any.whl (17 kB)
  Using cached banal-1.0.6-py2.py3-none-any.whl (6.1 kB)
  Using cached python_editor-1.0.4-py3-none-any.whl (4.9 kB)


In [4]:
!pip install bs4

###  Loading Required Pacakges

In [26]:
import requests # to get assess to the url & retriet data from url
import json # to convert scrapped data into JSON file
import re # for regular expression
from bs4 import BeautifulSoup as bs # for web scrapping
# import dataset
import csv

### Web Scrapping

##### Step 1 - Get url  & session request for browser

In [27]:
# Creating Dataset into Mongodb / SQLite
# db = dataset.connect('sqlite:/// reviews.db')

review_url = 'https://www.amazon.com/hz/reviews-render/ajax/reviews/get/'
product_id = '1449355730'
session = requests.Session()
session.headers.update({'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) \
Chrome/88.0.4324.190 Safari/537.36'})
url = 'https://www.amazon.com/product-reviews/{}/'.format(product_id)
session.get(url)


def parse_reviews(reply):
    reviews = []
    for fragment in reply.split('&&&'):
        if not fragment.strip():
            continue
        json_fragment = json.loads(fragment)
        if json_fragment[0] != 'append':
            continue
        html_soup = bs(json_fragment[2], 'html.parser')
        div = html_soup.find('div', {'data-hook':'review'})
        if not div:
            continue
        review_id = div.get('id')
        
        # find & clean the rating : 
        review_classes = ' '.join(html_soup.find(class_ = 'review-rating').get('class'))
        rating = re.search('a-star-(\d+)', review_classes).group(1)
        title = html_soup.find(class_='review-title').get_text(strip = True)
        review = html_soup.find(class_='review-text').get_text(strip = True)
        
        reviews.append({'review_id' : review_id,
                      'rating' : rating,
                      'title' : title,
                      'review' : review})
    return reviews


def get_reviews(product_id, page):
    data = {
        'sortBy' : '',
        'reveiwerType' : 'all_reviews',
        'formatType' : '',
        'mediaType' : '',
        'filterByStar' : 'all_stars',
        'pageNumber' : page,
        'filterByKeyword' : '',
        'shouldAppend' : 'undefined',
        'deviceType' : 'desktop',
        'reftag' : 'cm_cr_getr_d_paging_btm_{}'.format(page),
        'pageSize' : 15,
        'asin' : product_id,
        'scope' : 'reviewsAjax1'
    }
    r = session.post(review_url + 'ref=' + data['reftag'], data = data)
    reviews = parse_reviews(r.text)
    return reviews

page = 1
while True:
    print("Scrapping page", page)
    reviews = get_reviews(product_id, page)
    if not reviews:
        break
    for review in reviews:
        print('- Rating : {} \n Title: {} \n Review: \n{} \n'. format(review['rating'], review['title'], review['review']))
#         db['reviews'].upsert(review, ['review_id'])
    page += 1

Scrapping page 1
- Rating : 5 
 Title: Best Python book for a beginner 
 Review: 
OK. I'm a true beginner; seriously. I'm 35 yo English teacher who can easily create a WordPress site, or a Moodle course. Other than that, I know nothing of programming. HTML and CSS is easy. But all I know is copy-pasting code from CSS generators. Sure something has stick about CSS and HTML, but nothing about programming. I was terrified to even try.Until I met Python.It was love at first sight the moment I saw its syntax. It was like reading English. I felt like New in the Matrix. So I started watching tutorials on YouTube, taking courses on Udemy, and Lynda.com, but none of them provided insightful information about the ins and outs of Python; you know, like 'the devil's in the details' kind of stuff. (Sure there are some programmers who create video tutorials on YouTube but they cover only small parts of Python programming in depth and many of them left me wanting more.That's until I decided to google

- Rating : 5 
 Title: This is the real deal.  The full Python experience 
 Review: 
It's rare to find a book that so thoroughly covers a topic that you might not need to buy another Python book.  This book is one of those.  It is a MASSIVE guide to all things Python.  Check the pics for just how big this book really is.I'm not a programmer by trade so I can't comment on the technical aspects of Python.  It's more of a hobby for me but after working through the first few chapters, I'm hooked.Also, note the version numbers on the cover.  A pic of those are included as well.This isn't a book you'll finish in a day, a week, or even a month.  I suspect it will become a regular customer on my desk for months and months. 

- Rating : 1 
 Title: Incredibly verbose and repetitve. 
 Review: 
Lutz is trying to give you everything you ever need on just about any topic before he presents you with meat and potatoes. I just read two convoluted and not particularly exact chapters on object oriented pa

- Rating : 4 
 Title: Great Book but, geez, 8-point type? 
 Review: 
This is a 1,500 page, 2-3/4-inch thick book set, I would say, in 8 point type. I'm an old guy, and I'm having a devil of a time reading it. Because I like to type the code samples as I read, i'm having a lot of trouble getting the open book to stand upright in my copy holder so I can type the code as I go. I'm sure Mr. Lutz knows what he's talking about, and I am confident that I will learn to code in Python, but it's going to be a long time before I can make my way through it at the three or four pages a day I've managed so far to work on it. 

- Rating : 5 
 Title: My favorite Python bible 
 Review: 
If you want a quick overview of how to code Python, read the 'Tutorial' on the Python documentation website over the weekend. But after that, invest in this book because it gives the 'deep-dive' full and formal story. It will make you a real Python developer over time. I appreciate its level of detail and dedication to 

- Rating : 1 
 Title: Unfortunate purchase 
 Review: 
Perfect introduction to Python if you already everything there is to know about Python.I was hoping that the book was long because it would fill in all the cracks of my knowledge. Nope. Treading through the foggy doldrum of the author’s writing is a nightmare.Most people are not good teachers, this author is no exception. It is an insanely out of touch book.Eventually there will be such a huge sea of garbage resources available to us that it will be like having no resources at all. 

- Rating : 3 
 Title: There's a lot of information in here... 
 Review: 
...but a lot of it is repeated over and over.  I'm not going to accuse the author of trying to stretch the material to make the book bigger, but after the fifth or sixth repetition it can get pretty tedious.  I also bought his book, "Programming Python", and I'm hoping that it is a little more direct. 

- Rating : 5 
 Title: Huge book, worth every penny 
 Review: 
I've read, or sho

- Rating : 5 
 Title: Good introduction for  serious python code  -  not recommend as an intro to programming book. 
 Review: 
Packed with many useful examples and technical explanations for accomplishing useful things in the Python language. Well written and comprehensive but very readable if you're willing to get your hands dirty with coding; this book like most In this series are not for armchair reading.  Great when you're deep inside the code with something you're trying to accomplish. 

- Rating : 3 
 Title: Not perfect, Absolutely necessary reading 
 Review: 
It wouldn't be fair to mark it lower, but the cover and binding is pathetic! The publisher should be embarrassed: the spine splits when you open the book and lay it flat on a table. It was disappointing but at 1,540 pages you would think they'd make more of an effort. If you remove the binding problem this book is a Solid 4 stars! If it were easier to use as a reference, it would be 5 stars.If you're serious about learning 

...

- Rating : 5 
 Title: Comprehensive, but probably not for new programmers 
 Review: 
This book is not for the faint-hearted. It's not a quick introduction to the language, or a "Learn Python in X Minutes", or a light-hearted foray into what "if" statements and "loops" are. At 1600 pages long, this book is an incredibly in-depth, comprehensive dive into the fundamentals of the language. It'll teach you everything you'll ever need to know about Python, and some you might never use (there's a full two-page section on function annotations and why they're useful, for example). Learning Python documents an in-person Python course that Mark Lutz has been teaching for twenty years, and it shows: his knowledge of the language is almost unmatched.To tell you the truth, I probably wouldn't recommend the book as a person's first programming book, or, for that matter, their first Python book. The sheer density of the text will probably put a lot of first programmers off. However, for those who wish

- Rating : 4 
 Title: this book is an excellent bridge up to intermediate Python 
 Review: 
As others have said, this book is not suited for the absolute beginner. However, if you spend a little while on introduction youtube tutorials learning the syntax beforehand, this book is an excellent bridge up to intermediate Python. 

- Rating : 5 
 Title: Great 
 Review: 
Great 

- Rating : 3 
 Title: Good butu repetitious 
 Review: 
Could do with a good edit to cut out repetition. The book rather grinds through the subject.However, it does what it sets out to do as long as you don't mind going slowly - among the words there are many nuggets. 

- Rating : 2 
 Title: Not good for beginners 
 Review: 
I've seen better books, In my opinion it's not very organised. If you are new to python or programming, there are better much better books to read than this. 

- Rating : 4 
 Title: Good book for new learners 
 Review: 
This is a good book for beginners at python, messing with python started by lo

- Rating : 5 
 Title: Un Diccionario de Python con la D mayúscula 
 Review: 
Es un verdadero Diccionario, se encuentra casi todo lo relacionado con el lenguaje Python, aunque en mi opinión carece de explicación mas exhaustiva y detallada sobre el uso real de la memoria, conceptos funcionales y sus características mas avanzadas, contiene ejemplos  en todos sus capítulos y permite aprender buenas practicas.Entiendo que tiene un propósito general y por esto creo que es un libro muy completo en una etapa de aprendizaje. Aunque también es ideal para cualquiera que dese utilizar Python y quedarse con el. 

- Rating : 5 
 Title: Full Python training, Python reference 
 Review: 
You can also see this book as an encyclopedia on the subject. This is not a book that gives programming tricks and shortcut for that you need Programming Python from the same Author. If your goal is to learn Python, this book will take there and then some… Not for the faint of heart tho with its 1590 pages. Enjoy. 

- 

- Rating : 5 
 Title: Great for learning Python 
 Review: 
Mark knows the topic and explains it well. This is a good book for learning Python. But if you're looking for a reference manual, look at his pocket reference instead. 

- Rating : 5 
 Title: Definitely worth buying. 
 Review: 
The book is well written and covers more depth than I first expected. I have been out of the programming field for a decade, and it's nice to have clear well written explanations of the language. A touch of dry humour is great for me as well. 

- Rating : 5 
 Title: This is a comprehensive introduction to Python. In fact ... 
 Review: 
This is a comprehensive introduction to Python. In fact this is probably the most comprehensive book on python 2.x and 3.x. You will probably not need another book as an accompaniment. 

- Rating : 5 
 Title: python delivery 
 Review: 
arrivato in condizioni eccellenti! molto utile sia per chi vuole cominciare a programmare da 0 sia per coloro che vogliano approfondire pyt

- Rating : 1 
 Title: Laborious, boring and far too verbose 
 Review: 
It's frustrating to find an author with such a wealth of knowledge who is unable to communicate it in a way which is easily digestible. There are simply too many unnecessary words in this book. It makes finding the content you want difficult, and learning from the content both laborious and boring. This book would benefit from a ruthless editor who could chop it down to half or even a third of the size whilst losing little of the important concepts of python. 

- Rating : 2 
 Title: Nicht schlecht... 
 Review: 
...aber etwas langatmig. Entweder hast du schon Pragrammierkenntnisse von anderen Sprachen her, dann nerven dich die ausführlichen Vergleiche mit Java & Co. nur - oder du hast keine, dann verstehst du diese Passagen sowieso nicht. Wenn man schon ein paar Jahre mit mehreren anderen Sprachen sein Geld verdient hat und möchte jetzt noch Python dazu lernen, braucht man eher ein Buch, dass gerade aus beschreibt, w